<a href="https://colab.research.google.com/github/aloiswirth/handson-ml2/blob/master/12b_Ragged_tensors_addon_to_chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ragged tensors

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/ragged_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Setup


In [2]:
!pip install -q tf_nightly

     |████████████████████████████████| 322.7MB 48kB/s 
     |████████████████████████████████| 460kB 49.8MB/s 
     |████████████████████████████████| 6.8MB 47.9MB/s 


In [3]:
import math
import tensorflow as tf

## Overview

Your data comes in many shapes; your tensors should too.
*Ragged tensors* are the TensorFlow equivalent of nested variable-length
lists. They make it easy to store and process data with non-uniform shapes,
including:

*   Variable-length features, such as the set of actors in a movie.
*   Batches of variable-length sequential inputs, such as sentences or video
    clips.
*   Hierarchical inputs, such as text documents that are subdivided into
    sections, paragraphs, sentences, and words.
*   Individual fields in structured inputs, such as protocol buffers.

### What you can do with a ragged tensor
Ragged tensors are supported by more than a hundred TensorFlow operations, including math operations (such as tf.add and tf.reduce_mean), array operations (such as tf.concat and tf.tile), string manipulation ops (such as tf.substr), control flow operations (such as tf.while_loop and tf.map_fn), and many others:

In [4]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["So", "long"], ["thanks", "for", "all", "the", "fish"]])

There are also a number of methods and operations that are specific to ragged tensors, including factory methods, conversion methods, and value-mapping operations. For a list of supported ops, see the **`tf.ragged` package documentation**.

Ragged tensors are supported by many TensorFlow APIs, including [Keras](https://www.tensorflow.org/guide/keras), [Datasets](https://www.tensorflow.org/guide/data), [tf.function](https://www.tensorflow.org/guide/function), [SavedModels](https://www.tensorflow.org/guide/saved_model), and [tf.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord).  For more information, see the section on **TensorFlow APIs** below.

As with normal tensors, you can use Python-style indexing to access specific slices of a ragged tensor. For more information, see the section on **Indexing** below.

In [5]:
digits

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], []]>

In [6]:
print(digits[3])
digits[0]

tf.Tensor([6], shape=(1,), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 1, 4, 1], dtype=int32)>

In [7]:
print(digits[0])

tf.Tensor([3 1 4 1], shape=(4,), dtype=int32)


In [8]:
print(digits[:, :2])

<tf.RaggedTensor [[3, 1], [], [5, 9], [6], []]>


In [9]:
print(digits[:, -2:])

<tf.RaggedTensor [[4, 1], [], [9, 2], [6], []]>


In [10]:
# Adding 3 to all numbers in the tensor:
print(digits + 3) 

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>


In [11]:
print(tf.add(digits, 3))
# this adds 3 to all values

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>


In [12]:
# Now calculate the average
print(tf.reduce_mean(digits, axis=1))

tf.Tensor([2.25              nan 5.33333333 6.                nan], shape=(5,), dtype=float64)


In [13]:
# Conatenation of two Ragged Tensors:
print(tf.concat([digits, [[5, 3]]], axis=0))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [5, 3]]>


In [14]:
# Duplicate each component of a ragged tensor:
print(tf.tile(digits, [4, 1]))
print(tf.tile(digits, [1, 3]))
print(tf.tile(digits, [2, 3]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], [], [3, 1, 4, 1], [], [5, 9, 2], [6], []]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], []]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], [], [3, 1, 4, 1, 3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2, 5, 9, 2], [6, 6, 6], []]>


In [15]:
print(tf.map_fn(tf.math.square, digits))
# print(tf.map_fn(tf.math.exp(3.0) , digits))  ## doesn't work

<tf.RaggedTensor [[9, 1, 16, 1], [], [25, 81, 4], [36], []]>


In [16]:
words

<tf.RaggedTensor [[b'So', b'long'], [b'thanks', b'for', b'all', b'the', b'fish']]>

In [17]:
print(words[0])
words[1]

tf.Tensor([b'So' b'long'], shape=(2,), dtype=string)


<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'thanks', b'for', b'all', b'the', b'fish'], dtype=object)>

In [18]:
# substring operations on ragged string tensors
print(tf.strings.substr(words, 0, 2))
print(tf.strings.substr(words, 2, 4))

<tf.RaggedTensor [[b'So', b'lo'], [b'th', b'fo', b'al', b'th', b'fi']]>
<tf.RaggedTensor [[b'', b'ng'], [b'anks', b'r', b'l', b'e', b'sh']]>


Adding of ragged tensors (using +) can only be executed if the both tensors are of some type in all components. 

Excample [6] + [8, 9] does not work. See next example failing in the second line.

Error: Unable to broadcast: dimension size mismatch in dimension'
1
b'lengths='
4, 0, 3, 2, 0
b'dim_size='
4, 0, 3, 1, 0


In [19]:
print(digits + tf.ragged.constant([[1, 2, 3, 4], [], [5, 6, 7], [8], []]))

<tf.RaggedTensor [[4, 3, 7, 5], [], [10, 15, 9], [14], []]>


In [20]:
try:
  digits + tf.ragged.constant([[1, 2, 3, 4], [], [5, 6, 7], [8, 9], []])
except ValueError as exception:
  print(exception)
# except InvalidArgumentError as exception2:
#   print(exception2)
except:
  print("something else went wrong")

something else went wrong


If you need to perform an elementwise transformation to the values of a RaggedTensor, you can use tf.ragged.map_flat_values, which takes a function plus one or more arguments, and applies the function to transform the RaggedTensor's values.

In [21]:
times_two_plus_one = lambda x: x * 2 + 1
print(tf.ragged.map_flat_values(times_two_plus_one, digits))

<tf.RaggedTensor [[7, 3, 9, 3], [], [11, 19, 5], [13], []]>


In [22]:
exponent_three_plus_one = lambda x: x ** 3 + 1
print(tf.ragged.map_flat_values(exponent_three_plus_one, digits))

<tf.RaggedTensor [[28, 2, 65, 2], [], [126, 730, 9], [217], []]>


Conversion of ragged tensor to list of numpy nd array:

In [23]:
my_converted_list = digits.to_list()
print(my_converted_list)
for item in my_converted_list:
  print(item)

[[3, 1, 4, 1], [], [5, 9, 2], [6], []]
[3, 1, 4, 1]
[]
[5, 9, 2]
[6]
[]


In [24]:
np_converted_tensors = digits.numpy()
for my_array in np_converted_tensors:
  print(my_array)

[3 1 4 1]
[]
[5 9 2]
[6]
[]


### Constructing a ragged tensor
The simplest way to construct a ragged tensor is using `tf.ragged.constant`, which builds the `RaggedTensor` corresponding to a given nested Python `list` or numpy `array`:

In [25]:
sentences = tf.ragged.constant([
    ["Let's", "build", "some", "ragged", "tensors", "!"],
    ["We", "can", "use", "tf.ragged.constant", "."]])
print(sentences)
for i in sentences:
  print(i)

<tf.RaggedTensor [[b"Let's", b'build', b'some', b'ragged', b'tensors', b'!'], [b'We', b'can', b'use', b'tf.ragged.constant', b'.']]>
tf.Tensor([b"Let's" b'build' b'some' b'ragged' b'tensors' b'!'], shape=(6,), dtype=string)
tf.Tensor([b'We' b'can' b'use' b'tf.ragged.constant' b'.'], shape=(5,), dtype=string)


In [26]:
list_sentences = sentences.to_list()
list_sentences

[[b"Let's", b'build', b'some', b'ragged', b'tensors', b'!'],
 [b'We', b'can', b'use', b'tf.ragged.constant', b'.']]

In [27]:
paragraphs = tf.ragged.constant([
    [['I', 'have', 'a', 'cat'], ['His', 'name', 'is', 'Mat']],
    [['Do', 'you', 'want', 'to', 'come', 'visit'], ["I'm", 'free', 'tomorrow']],
])
print(paragraphs)

<tf.RaggedTensor [[[b'I', b'have', b'a', b'cat'], [b'His', b'name', b'is', b'Mat']], [[b'Do', b'you', b'want', b'to', b'come', b'visit'], [b"I'm", b'free', b'tomorrow']]]>


In [28]:
list_paragraphs = paragraphs.to_list()
list_paragraphs

[[[b'I', b'have', b'a', b'cat'], [b'His', b'name', b'is', b'Mat']],
 [[b'Do', b'you', b'want', b'to', b'come', b'visit'],
  [b"I'm", b'free', b'tomorrow']]]

#### Alternative Representations of Ragged Tensors
Ragged tensors can also be constructed by pairing flat *values* tensors with
*row-partitioning* tensors indicating how those values should be divided into
rows, using factory classmethods such as 
* `tf.RaggedTensor.from_value_rowids`,
* `tf.RaggedTensor.from_row_lengths`, and
* `tf.RaggedTensor.from_row_splits`.

##### 1.) `tf.RaggedTensor.from_value_rowids`
If you know which row each value belongs in, then you can build a `RaggedTensor` using a `value_rowids` row-partitioning tensor:

![value_rowids](https://www.tensorflow.org/images/ragged_tensors/value_rowids.png)

In [29]:
print(tf.RaggedTensor.from_value_rowids(
    values=[3, 1, 4, 1, 5, 9, 2],
    value_rowids=[0, 0, 0, 0, 2, 2, 3]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2]]>


##### 2.) `tf.RaggedTensor.from_row_lengths`

If you know how long each row is, then you can use a `row_lengths` row-partitioning tensor:

![row_lengths](https://www.tensorflow.org/images/ragged_tensors/row_lengths.png)

In [30]:
print(tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_lengths=[4, 0, 2, 1]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2]]>


##### 3.) `tf.RaggedTensor.from_row_splits`

If you know the index where each row starts and ends, then you can use a `row_splits` row-partitioning tensor:

![row_splits](https://www.tensorflow.org/images/ragged_tensors/row_splits.png)

In [31]:
print(tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_splits=[0, 4, 4, 6, 7, 7]))

<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9], [2], []]>


See the `tf.RaggedTensor` class documentation for a full list of factory methods.

Note: By default, these factory methods add assertions that the row partition tensor is well-formed and consistent with the number of values.  The `validate=False` parameter can be used to skip these checks if you can guarantee that the inputs are well-formed and consistent.

### What you can store in a ragged tensor

As with normal `Tensor`s, the values in a `RaggedTensor` must all have the same
type; and the values must all be at the same nesting depth (the *rank* of the
tensor):

In [32]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]))  # ok: type=string, rank=2

<tf.RaggedTensor [[b'Hi'], [b'How', b'are', b'you']]>


In [33]:
print(tf.ragged.constant([[[1, 2], [3]], [[4, 5]]]))        # ok: type=int32, rank=3

<tf.RaggedTensor [[[1, 2], [3]], [[4, 5]]]>


In [34]:
try:
  tf.ragged.constant([["one", "two"], [3, 4]])              # bad: multiple types
except ValueError as exception:
  print(exception)

Can't convert Python sequence with mixed types to Tensor.


In [35]:
try:
  tf.ragged.constant(["A", ["B", "C"]])                     # bad: multiple nesting depths
except ValueError as exception:
  print(exception)

all scalar values must have the same nesting depth


## Example use case

The following example demonstrates how `RaggedTensor`s can be used to construct
and combine unigram and bigram embeddings for a batch of variable-length
queries, using special markers for the beginning and end of each sentence.
For more details on the ops used in this example, see the `tf.ragged` package documentation.

In [36]:
queries = tf.ragged.constant([['Who', 'is', 'today', 'Dan', 'Smith'],
                              ['Pause'],
                              ['Will', 'it', 'rain', 'later', 'today']])
queries

<tf.RaggedTensor [[b'Who', b'is', b'today', b'Dan', b'Smith'], [b'Pause'], [b'Will', b'it', b'rain', b'later', b'today']]>

In [37]:
queries.shape

TensorShape([3, None])

In [38]:
# just to see how this works
len(queries.to_list())

3

In [39]:
# Create an embedding table.
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                       stddev=1.0 / math.sqrt(embedding_size)))

print(embedding_table)
'''
array([[ 0.7967023 ,  0.80937356, -0.933402  , -0.76543057],
       [-0.11630829,  0.14486092, -0.69844437, -0.6212817 ],
'''

<tf.Variable 'Variable:0' shape=(1024, 4) dtype=float32, numpy=
array([[-0.06978837,  0.19359063, -0.34617987,  0.11603856],
       [-0.22733235,  0.3751056 ,  0.14743423, -0.34628192],
       [ 0.01751955, -0.07190721,  0.7064819 ,  0.44548655],
       ...,
       [-0.9181837 ,  0.16131419, -0.20291993, -0.03120707],
       [ 0.39511496,  0.46536842,  0.03990752, -0.21964796],
       [ 0.10540861,  0.36131456, -0.34862486,  0.04821806]],
      dtype=float32)>


'\narray([[ 0.7967023 ,  0.80937356, -0.933402  , -0.76543057],\n       [-0.11630829,  0.14486092, -0.69844437, -0.6212817 ],\n'

In [40]:
math.sqrt(embedding_size)

2.0

In [41]:
for i in range(1024):
  for j in range(4):
    print(embedding_table[i , j])

tf.Tensor(-0.069788374, shape=(), dtype=float32)
tf.Tensor(0.19359063, shape=(), dtype=float32)
tf.Tensor(-0.34617987, shape=(), dtype=float32)
tf.Tensor(0.11603856, shape=(), dtype=float32)
tf.Tensor(-0.22733235, shape=(), dtype=float32)
tf.Tensor(0.3751056, shape=(), dtype=float32)
tf.Tensor(0.14743423, shape=(), dtype=float32)
tf.Tensor(-0.34628192, shape=(), dtype=float32)
tf.Tensor(0.017519554, shape=(), dtype=float32)
tf.Tensor(-0.07190721, shape=(), dtype=float32)
tf.Tensor(0.7064819, shape=(), dtype=float32)
tf.Tensor(0.44548655, shape=(), dtype=float32)
tf.Tensor(0.7555735, shape=(), dtype=float32)
tf.Tensor(0.12065971, shape=(), dtype=float32)
tf.Tensor(-0.6438394, shape=(), dtype=float32)
tf.Tensor(0.11610095, shape=(), dtype=float32)
tf.Tensor(0.44473118, shape=(), dtype=float32)
tf.Tensor(0.5352465, shape=(), dtype=float32)
tf.Tensor(0.42331794, shape=(), dtype=float32)
tf.Tensor(0.9026192, shape=(), dtype=float32)
tf.Tensor(-0.2111841, shape=(), dtype=float32)
tf.Tensor(0

In [42]:
# Look up the embedding for each word.
# word_buckets contains integers reqresenting the word strings
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
# word_embeddings contain a tensor representation of word bucket within the embedding table
word_embeddings = tf.nn.embedding_lookup(embedding_table, word_buckets)     # ①
print(word_buckets)
print(word_embeddings)

<tf.RaggedTensor [[633, 768, 547, 237, 309], [28], [847, 998, 635, 20, 547]]>
<tf.RaggedTensor [[[-0.20747829973697662, 0.42940646409988403, -0.38184067606925964, 0.0786549374461174], [0.2910548746585846, 0.7162835597991943, -0.6078064441680908, -0.18328335881233215], [-0.6401904225349426, -0.3893193304538727, -0.0678597018122673, -0.24108487367630005], [-0.3347620368003845, 0.840153694152832, -0.5106130838394165, 0.17591872811317444], [0.4736846387386322, 0.509670615196228, 0.09753894060850143, -0.11929423362016678]], [[-0.29218846559524536, -0.671757161617279, 0.34031254053115845, -0.490825355052948]], [[0.6114388704299927, 0.10289561748504639, -0.3564261198043823, 0.1682262122631073], [0.37532004714012146, 0.2964819669723511, 0.1102650985121727, 0.655766487121582], [-0.18240754306316376, 0.38665902614593506, -0.6780259609222412, 0.0974561795592308], [-0.7859180569648743, -0.585277795791626, 0.567176878452301, -0.7651062607765198], [-0.6401904225349426, -0.3893193304538727, -0.067859

In [43]:
# Add markers to the beginning and end of each sentence.
marker = tf.fill([queries.nrows(), 1], '#')
marker

<tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'#'],
       [b'#'],
       [b'#']], dtype=object)>

In [44]:
padded = tf.concat([marker, queries, marker], axis=1)                       # ②
print(padded)

<tf.RaggedTensor [[b'#', b'Who', b'is', b'today', b'Dan', b'Smith', b'#'], [b'#', b'Pause', b'#'], [b'#', b'Will', b'it', b'rain', b'later', b'today', b'#']]>


In [45]:
# Build word bigrams & look up embeddings.
bigrams = tf.strings.join([padded[:, :-1], padded[:, 1:]], separator='+')   # ③
print(bigrams)

<tf.RaggedTensor [[b'#+Who', b'Who+is', b'is+today', b'today+Dan', b'Dan+Smith', b'Smith+#'], [b'#+Pause', b'Pause+#'], [b'#+Will', b'Will+it', b'it+rain', b'rain+later', b'later+today', b'today+#']]>


In [46]:
bigram_buckets = tf.strings.to_hash_bucket_fast(bigrams, num_buckets)
print(bigram_buckets)


<tf.RaggedTensor [[939, 196, 460, 372, 281, 176], [597, 723], [853, 293, 853, 704, 749, 372]]>


In [47]:
bigram_embeddings = tf.nn.embedding_lookup(embedding_table, bigram_buckets) # ④
print(bigram_embeddings)
print(bigram_embeddings.shape)

<tf.RaggedTensor [[[0.7246147394180298, 0.07524210214614868, 0.7339106202125549, -0.14881475269794464], [-0.5499006509780884, 0.1553564816713333, 0.21857580542564392, -0.2269831746816635], [0.8145033717155457, 0.30890095233917236, 0.9361680150032043, 0.05983081832528114], [-0.33002403378486633, 0.01935429871082306, -0.49667036533355713, 0.4037536680698395], [-0.10038954764604568, -0.13476340472698212, 0.9296860694885254, 0.6053377389907837], [-0.7876598834991455, -0.39777421951293945, -0.7151308059692383, -0.1403372436761856]], [[0.5823306441307068, 0.2086392492055893, -0.20576050877571106, 0.6391780972480774], [0.12652646005153656, 0.1980108618736267, -0.4506562352180481, 0.3205668330192566]], [[-0.4303804636001587, 0.22454535961151123, -0.6487599015235901, 0.25548532605171204], [0.07703178375959396, -0.1638236939907074, -0.5595942139625549, -0.05444108322262764], [-0.4303804636001587, 0.22454535961151123, -0.6487599015235901, 0.25548532605171204], [-0.6146142482757568, -0.65589213371

The above print shows 3, None, 4. 3 is the number of sentences, 'None' is indicating that the number of words per sentance varies, and thus the dimension is ragged. 4 indicates the embedding size. 

In [48]:
# Find the average embedding for each sentence
all_embeddings = tf.concat([word_embeddings, bigram_embeddings], axis=1)    # ⑤
print(all_embeddings.shape)
all_embeddings

(3, None, 4)


<tf.RaggedTensor [[[-0.20747829973697662, 0.42940646409988403, -0.38184067606925964, 0.0786549374461174], [0.2910548746585846, 0.7162835597991943, -0.6078064441680908, -0.18328335881233215], [-0.6401904225349426, -0.3893193304538727, -0.0678597018122673, -0.24108487367630005], [-0.3347620368003845, 0.840153694152832, -0.5106130838394165, 0.17591872811317444], [0.4736846387386322, 0.509670615196228, 0.09753894060850143, -0.11929423362016678], [0.7246147394180298, 0.07524210214614868, 0.7339106202125549, -0.14881475269794464], [-0.5499006509780884, 0.1553564816713333, 0.21857580542564392, -0.2269831746816635], [0.8145033717155457, 0.30890095233917236, 0.9361680150032043, 0.05983081832528114], [-0.33002403378486633, 0.01935429871082306, -0.49667036533355713, 0.4037536680698395], [-0.10038954764604568, -0.13476340472698212, 0.9296860694885254, 0.6053377389907837], [-0.7876598834991455, -0.39777421951293945, -0.7151308059692383, -0.1403372436761856]], [[-0.29218846559524536, -0.671757161617

In [49]:
avg_embedding = tf.reduce_mean(all_embeddings, axis=1)                      # ⑥
print(avg_embedding)

tf.Tensor(
[[-0.05877703  0.19386463  0.01235985  0.02397257]
 [ 0.13888955 -0.08836901 -0.10536807  0.15630652]
 [-0.19492674 -0.03264832 -0.26683104  0.11577435]], shape=(3, 4), dtype=float32)


![ragged_example](https://www.tensorflow.org/images/ragged_tensors/ragged_example.png)

## Ragged and uniform dimensions

A ***ragged dimension*** is a dimension whose slices may have different lengths. For example, the
inner (column) dimension of `rt=[[3, 1, 4, 1], [], [5, 9, 2], [6], []]` is
ragged, since the column slices (`rt[0, :]`, ..., `rt[4, :]`) have different
lengths. Dimensions whose slices all have the same length are called *uniform
dimensions*.

The outermost dimension of a ragged tensor is always uniform, since it consists
of a single slice (and so there is no possibility for differing slice
lengths).  The remaining dimensions may be either ragged or uniform. For
example, we might store the word embeddings for
each word in a batch of sentences using a ragged tensor with shape
`[num_sentences, (num_words), embedding_size]`, where the parentheses around
`(num_words)` indicate that the dimension is ragged.

![sent_word_embed](https://www.tensorflow.org/images/ragged_tensors/sent_word_embed.png)

Ragged tensors may have multiple ragged dimensions. For example, we could store
a batch of structured text documents using a tensor with shape `[num_documents,
(num_paragraphs), (num_sentences), (num_words)]` (where again parentheses are
used to indicate ragged dimensions).

As with `tf.Tensor`, the ***rank*** of a ragged tensor is its total number of dimensions (including both ragged and uniform dimensions).
A ***potentially ragged tensor*** is a value that might be
either a `tf.Tensor` or a `tf.RaggedTensor`.

When describing the shape of a RaggedTensor, ragged dimensions are conventionally indicated by
enclosing them in parentheses. For example, as we saw above, the shape of a 3-D
RaggedTensor that stores word embeddings for each word in a batch of sentences
can be written as `[num_sentences, (num_words), embedding_size]`.

The `RaggedTensor.shape` attribute returns a `tf.TensorShape` for a
ragged tensor, where ragged dimensions have size `None`:

In [50]:
tf.ragged.constant([["Hi"], ["How", "are", "you"]]).shape

TensorShape([2, None])

The method `tf.RaggedTensor.bounding_shape` can be used to find a tight
bounding shape for a given `RaggedTensor`:

In [51]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]).bounding_shape())

tf.Tensor([2 3], shape=(2,), dtype=int64)


## Ragged vs. sparse

A ragged tensor should *not* be thought of as a type of sparse tensor.  In particular, sparse tensors are *efficient encodings for tf.Tensor*, that model the same data in a compact format; but ragged tensor is an *extension to tf.Tensor*, that models an expanded class of data.  This difference is crucial when defining operations:

* Applying an op to a sparse or dense tensor should always give the same result.
* Applying an op to a ragged or sparse tensor may give different results.

As an illustrative example, consider how array operations such as `concat`,
`stack`, and `tile` are defined for ragged vs. sparse tensors. Concatenating
ragged tensors joins each row to form a single row with the combined length:

![ragged_concat](https://www.tensorflow.org/images/ragged_tensors/ragged_concat.png)


In [52]:
ragged_x = tf.ragged.constant([["John"], ["a", "big", "dog"], ["my", "cat"]])
ragged_y = tf.ragged.constant([["fell", "asleep"], ["barked"], ["is", "fuzzy"]])
print(tf.concat([ragged_x, ragged_y], axis=1))

<tf.RaggedTensor [[b'John', b'fell', b'asleep'], [b'a', b'big', b'dog', b'barked'], [b'my', b'cat', b'is', b'fuzzy']]>


But concatenating sparse tensors is equivalent to concatenating the corresponding dense tensors,
as illustrated by the following example (where Ø indicates missing values):

![sparse_concat](https://www.tensorflow.org/images/ragged_tensors/sparse_concat.png)

In [53]:
sparse_x = ragged_x.to_sparse()
print(sparse_x)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]], shape=(6, 2), dtype=int64), values=tf.Tensor([b'John' b'a' b'big' b'dog' b'my' b'cat'], shape=(6,), dtype=string), dense_shape=tf.Tensor([3 3], shape=(2,), dtype=int64))


In [54]:
sparse_y = ragged_y.to_sparse()
print(sparse_y)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [2 0]
 [2 1]], shape=(5, 2), dtype=int64), values=tf.Tensor([b'fell' b'asleep' b'barked' b'is' b'fuzzy'], shape=(5,), dtype=string), dense_shape=tf.Tensor([3 2], shape=(2,), dtype=int64))


In [55]:
sparse_result = tf.sparse.concat(sp_inputs=[sparse_x, sparse_y], axis=1)
print(tf.sparse.to_dense(sparse_result, ''))

tf.Tensor(
[[b'John' b'' b'' b'fell' b'asleep']
 [b'a' b'big' b'dog' b'barked' b'']
 [b'my' b'cat' b'' b'is' b'fuzzy']], shape=(3, 5), dtype=string)


For another example of why this distinction is important, consider the
definition of “the mean value of each row” for an op such as `tf.reduce_mean`.
For a ragged tensor, the mean value for a row is the sum of the
row’s values divided by the row’s width.
But for a sparse tensor, the mean value for a row is the sum of the
row’s values divided by the sparse tensor’s overall width (which is
greater than or equal to the width of the longest row).

## TensorFlow APIs


### Keras

[tf.keras](https://www.tensorflow.org/guide/keras) is TensorFlow's high-level API for building and training deep learning models. Ragged tensors may be passed as inputs to a Keras model by setting `ragged=True` on `tf.keras.Input` or `tf.keras.layers.InputLayer`.  Ragged tensors may also be passed between Keras layers, and returned by Keras models.  The following example shows a toy LSTM model that is trained using ragged tensors.

In [56]:
# Task: predict whether each sentence is a question or not.
sentences = tf.constant(
    ['What makes you think she is a witch?',
     'She turned me into a newt.',
     'A newt?',
     'Well, I got better.'])
is_question = tf.constant([True, False, True, False])

# Preprocess the input strings.
hash_buckets = 1000
words = tf.strings.split(sentences, ' ')
hashed_words = tf.strings.to_hash_bucket_fast(words, hash_buckets)
print(hashed_words)


<tf.RaggedTensor [[940, 203, 668, 387, 790, 320, 939, 185], [315, 515, 791, 181, 939, 787], [564, 205], [820, 180, 993, 739]]>


In [57]:
# Build the Keras model.
keras_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[None], dtype=tf.int64, ragged=True),
    tf.keras.layers.Embedding(hash_buckets, 16),
    tf.keras.layers.LSTM(32, use_bias=False),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation(tf.nn.relu),
    tf.keras.layers.Dense(1)
])

In [58]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          16000     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6144      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 23,233
Trainable params: 23,233
Non-trainable params: 0
_________________________________________________________________


In [59]:
keras_model.compile(loss='binary_crossentropy', optimizer='rmsprop')
keras_model.fit(hashed_words, is_question, epochs=5)
print(keras_model.predict(hashed_words))

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 1ms/step - loss: 5.0574
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 2.4491
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 1.8115
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 1.7083
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 1.6524
[[0.03264387]
 [0.01967335]
 [0.05139512]
 [0.01372125]]


In [60]:
keras_model.get_weights()

[array([[-0.02830984, -0.02507193,  0.04197246, ...,  0.04942912,
         -0.00542276, -0.03116726],
        [ 0.01810733, -0.04399847, -0.02813717, ..., -0.00629228,
          0.03634275,  0.04129019],
        [ 0.0284822 , -0.00483435, -0.02682203, ...,  0.00786133,
          0.03648994,  0.04026446],
        ...,
        [-0.04238371,  0.01198821,  0.0351469 , ...,  0.02114916,
          0.03330387,  0.04896982],
        [ 0.01841651,  0.01436785, -0.03312881, ...,  0.01360866,
          0.00329797,  0.00689076],
        [-0.00916864,  0.04768013, -0.0049474 , ..., -0.00048461,
          0.001994  ,  0.0483681 ]], dtype=float32),
 array([[ 0.18498406, -0.14529166, -0.06377826, ...,  0.08832143,
          0.0248252 , -0.02605689],
        [-0.12048881, -0.16305907,  0.14887865, ..., -0.1311625 ,
         -0.08203234,  0.19125193],
        [-0.10782951, -0.1665738 ,  0.15943243, ...,  0.1511478 ,
          0.10524347, -0.11786704],
        ...,
        [ 0.12437019,  0.17204283, -0.1

### tf.Example

[tf.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord) is a standard [protobuf](https://developers.google.com/protocol-buffers/) encoding for TensorFlow data.  Data encoded with `tf.Example`s often includes variable-length features.  For example, the following code defines a batch of four `tf.Example` messages with different feature lengths:

In [61]:
import google.protobuf.text_format as pbtext

def build_tf_example(s):
  return pbtext.Merge(s, tf.train.Example()).SerializeToString()

example_batch = [
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["red", "blue"]} } }
      feature {key: "lengths" value {int64_list {value: [7]} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["orange"]} } }
      feature {key: "lengths" value {int64_list {value: []} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["black", "yellow"]} } }
      feature {key: "lengths" value {int64_list {value: [1, 3]} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["green"]} } }
      feature {key: "lengths" value {int64_list {value: [3, 5, 2]} } } }''')                 
]

We can parse this encoded data using `tf.io.parse_example`, which takes a tensor of serialized strings and a feature specification dictionary, and returns a dictionary mapping feature names to tensors.  To read the variable-length features into ragged tensors, we simply use `tf.io.RaggedFeature` in the feature specification dictionary:

In [62]:
feature_specification = {
    'colors': tf.io.RaggedFeature(tf.string),
    'lengths': tf.io.RaggedFeature(tf.int64),
}
print(feature_specification)

print(feature_specification.values() )

{'colors': RaggedFeature(dtype=tf.string, value_key=None, partitions=(), row_splits_dtype=tf.int32, validate=False), 'lengths': RaggedFeature(dtype=tf.int64, value_key=None, partitions=(), row_splits_dtype=tf.int32, validate=False)}
dict_values([RaggedFeature(dtype=tf.string, value_key=None, partitions=(), row_splits_dtype=tf.int32, validate=False), RaggedFeature(dtype=tf.int64, value_key=None, partitions=(), row_splits_dtype=tf.int32, validate=False)])


In [63]:
feature_tensors = tf.io.parse_example(example_batch, feature_specification)
for name, value in feature_tensors.items():
  print("{}={}".format(name, value))

colors=<tf.RaggedTensor [[b'red', b'blue'], [b'orange'], [b'black', b'yellow'], [b'green']]>
lengths=<tf.RaggedTensor [[7], [], [1, 3], [3, 5, 2]]>


`tf.io.RaggedFeature` can also be used to read features with multiple ragged dimensions.  For details, see the [API documentation](https://www.tensorflow.org/api_docs/python/tf/io/RaggedFeature).

### Datasets

[tf.data](https://www.tensorflow.org/guide/data) is an API that enables you to build complex input pipelines from simple, reusable pieces.  Its core data structure is `tf.data.Dataset`, which represents a sequence of elements, in which each element consists of one or more components. 

In [64]:
# Helper function used to print datasets in the examples below.
def print_dictionary_dataset(dataset):
  for i, element in enumerate(dataset):
    print("Element {}:".format(i))
    for (feature_name, feature_value) in element.items():
      print('{:>14} = {}'.format(feature_name, feature_value))

#### Building Datasets with ragged tensors

Datasets can be built from ragged tensors using the same methods that are used to build them from `tf.Tensor`s or numpy `array`s, such as `Dataset.from_tensor_slices`:

In [65]:
dataset = tf.data.Dataset.from_tensor_slices(feature_tensors)
print_dictionary_dataset(dataset)

Element 0:
        colors = [b'red' b'blue']
       lengths = [7]
Element 1:
        colors = [b'orange']
       lengths = []
Element 2:
        colors = [b'black' b'yellow']
       lengths = [1 3]
Element 3:
        colors = [b'green']
       lengths = [3 5 2]


Note: Dataset.from_generator does not support ragged tensors yet, but support will be added soon.

#### Batching and unbatching Datasets with ragged tensors

Datasets with ragged tensors can be batched (which combines *n* consecutive elements into a single elements) using the `Dataset.batch` method.

In [66]:
batched_dataset = dataset.batch(2)
print_dictionary_dataset(batched_dataset)

Element 0:
        colors = <tf.RaggedTensor [[b'red', b'blue'], [b'orange']]>
       lengths = <tf.RaggedTensor [[7], []]>
Element 1:
        colors = <tf.RaggedTensor [[b'black', b'yellow'], [b'green']]>
       lengths = <tf.RaggedTensor [[1, 3], [3, 5, 2]]>


Conversely, a batched dataset can be transformed into a flat dataset using Dataset.unbatch.

In [67]:
unbatched_dataset = batched_dataset.unbatch()
print_dictionary_dataset(unbatched_dataset)

Element 0:
        colors = [b'red' b'blue']
       lengths = [7]
Element 1:
        colors = [b'orange']
       lengths = []
Element 2:
        colors = [b'black' b'yellow']
       lengths = [1 3]
Element 3:
        colors = [b'green']
       lengths = [3 5 2]


Batching Datasets with variable-length non-ragged tensors
If you have a Dataset that contains non-ragged tensors, and tensor lengths vary across elements, then you can batch those non-ragged tensors into ragged tensors by applying the dense_to_ragged_batch transformation:

In [68]:
non_ragged_dataset = tf.data.Dataset.from_tensor_slices([1, 5, 3, 2, 8])
print("Initial ", non_ragged_dataset)
non_ragged_dataset = non_ragged_dataset.map(tf.range)
print("Second" ,non_ragged_dataset)
batched_non_ragged_dataset = non_ragged_dataset.apply(
    tf.data.experimental.dense_to_ragged_batch(2))
print("now the dense to ragged dataset")
for element in batched_non_ragged_dataset:
  print(element)

Initial  <TensorSliceDataset shapes: (), types: tf.int32>
Second <MapDataset shapes: (None,), types: tf.int32>
now the dense to ragged dataset
<tf.RaggedTensor [[0], [0, 1, 2, 3, 4]]>
<tf.RaggedTensor [[0, 1, 2], [0, 1]]>
<tf.RaggedTensor [[0, 1, 2, 3, 4, 5, 6, 7]]>


#### Transforming Datasets with ragged tensors

Ragged tensors in Datasets can also be created or transformed using `Dataset.map`.

In [69]:
def transform_lengths(features):
  return {
      'mean_length': tf.math.reduce_mean(features['lengths']),
      'length_ranges': tf.ragged.range(features['lengths'])}
transformed_dataset = dataset.map(transform_lengths)
print_dictionary_dataset(transformed_dataset)

Element 0:
   mean_length = 7
 length_ranges = <tf.RaggedTensor [[0, 1, 2, 3, 4, 5, 6]]>
Element 1:
   mean_length = 0
 length_ranges = <tf.RaggedTensor []>
Element 2:
   mean_length = 2
 length_ranges = <tf.RaggedTensor [[0], [0, 1, 2]]>
Element 3:
   mean_length = 3
 length_ranges = <tf.RaggedTensor [[0, 1, 2], [0, 1, 2, 3, 4], [0, 1]]>


### tf.function

[tf.function](https://www.tensorflow.org/guide/function) is a decorator that precomputes TensorFlow graphs for Python functions, which can substantially improve the performance of your TensorFlow code.  Ragged tensors can be used transparently with `@tf.function`-decorated functions.  For example, the following function works with both ragged and non-ragged tensors:

In [70]:
@tf.function
def make_palindrome(x, axis):
  return tf.concat([x, tf.reverse(x, [axis])], axis)

In [71]:
# Normal tensor:
make_palindrome(tf.constant([[1, 2], [3, 4], [5, 6]]), axis=1)

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 2, 2, 1],
       [3, 4, 4, 3],
       [5, 6, 6, 5]], dtype=int32)>

In [73]:
# ragged tensor with decoration @tf.function
make_palindrome(tf.ragged.constant([[1, 2], [3], [4, 5, 6]]), axis=1)

<tf.RaggedTensor [[1, 2, 2, 1], [3, 3], [4, 5, 6, 6, 5, 4]]>

If you wish to explicitly specify the `input_signature` for the `tf.function`, then you can do so using `tf.RaggedTensorSpec`.

In [74]:
@tf.function(
    input_signature=[tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int32)])
def max_and_min(rt):
  return (tf.math.reduce_max(rt, axis=-1), tf.math.reduce_min(rt, axis=-1))

max_and_min(tf.ragged.constant([[1, 2], [3], [4, 5, 6]]))

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 6], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 3, 4], dtype=int32)>)

#### Concrete functions

[Concrete functions](https://www.tensorflow.org/guide/concrete_function) encapsulate individual traced graphs that are built by `tf.function`.  Starting with TensorFlow 2.3 (and in `tf-nightly`), ragged tensors can be used transparently with concrete functions.


In [75]:
# Preferred way to use ragged tensors with concrete functions (TF 2.3+):
try:
  @tf.function
  def increment(x):
    return x + 1

  rt = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
  cf = increment.get_concrete_function(rt)
  print(cf(rt))
except Exception as e:
  print(f"Not supported before TF 2.3: {type(e)}: {e}")

<tf.RaggedTensor [[2, 3], [4], [5, 6, 7]]>


### SavedModels

A [SavedModel](https://www.tensorflow.org/guide/saved_model) is a serialized TensorFlow program, including both weights and computation.  It can be built from a Keras model or from a custom model.  In either case, ragged tensors can be used transparently with the functions and methods defined by a SavedModel.

#### Example: saving a Keras model

In [76]:
import tempfile

keras_module_path = tempfile.mkdtemp()
tf.saved_model.save(keras_model, keras_module_path)
imported_model = tf.saved_model.load(keras_module_path)
imported_model(hashed_words)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpt4o9ilcy/assets


<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.03264387],
       [0.01967335],
       [0.05139512],
       [0.01372125]], dtype=float32)>

#### Example: saving a custom model

In [77]:
class CustomModule(tf.Module):
  def __init__(self, variable_value):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(variable_value)

  @tf.function
  def grow(self, x):
    return x * self.v

module = CustomModule(100.0)

# Before saving a custom model, we must ensure that concrete functions are
# built for each input signature that we will need.
module.grow.get_concrete_function(tf.RaggedTensorSpec(shape=[None, None],
                                                      dtype=tf.float32))

custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)
imported_model.grow(tf.ragged.constant([[1.0, 4.0, 3.0], [2.0]]))

INFO:tensorflow:Assets written to: /tmp/tmplpsn0o9g/assets


<tf.RaggedTensor [[100.0, 400.0, 300.0], [200.0]]>

Note: SavedModel [signatures](https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export) are concrete functions.  As discussed in the section on Concrete Functions above, ragged tensors are only handled correctly by concrete functions starting with TensorFlow 2.3 (and in `tf_nightly`).  If you need to use SavedModel signatures in a previous version of TensorFlow, then we recommend decomposing the ragged tensor into its component tensors.

## Overloaded operators

The `RaggedTensor` class overloads the standard Python arithmetic and comparison
operators, making it easy to perform basic elementwise math:

In [78]:
x = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
y = tf.ragged.constant([[1, 1], [2], [3, 3, 3]])
print(x + y)

<tf.RaggedTensor [[2, 3], [5], [7, 8, 9]]>


For a discussion of more advanced cases, see the section on
**Broadcasting**.

Ragged tensors overload the same set of operators as normal `Tensor`s: the unary
operators `-`, `~`, and `abs()`; and the binary operators `+`, `-`, `*`, `/`,
`//`, `%`, `**`, `&`, `|`, `^`, `==`, `<`, `<=`, `>`, and `>=`.


## Indexing

Ragged tensors support Python-style indexing, including multidimensional
indexing and slicing. The following examples demonstrate ragged tensor indexing
with a 2-D and a 3-D ragged tensor.

### Indexing examples: 2D ragged tensor

In [79]:
queries = tf.ragged.constant(
    [['Who', 'is', 'George', 'Washington'],
     ['What', 'is', 'the', 'weather', 'tomorrow'],
     ['Goodnight']])

In [80]:
print(queries[1])                 # A single query

tf.Tensor([b'What' b'is' b'the' b'weather' b'tomorrow'], shape=(5,), dtype=string)


In [81]:
print(queries[1, 2])              # a single word

tf.Tensor(b'the', shape=(), dtype=string)


In [82]:
print(queries[1: ])               # eveeything but the first row 

<tf.RaggedTensor [[b'What', b'is', b'the', b'weather', b'tomorrow'], [b'Goodnight']]>


In [83]:
print(queries[:, :3])             # all rows until th ethird word

<tf.RaggedTensor [[b'Who', b'is', b'George'], [b'What', b'is', b'the'], [b'Goodnight']]>


In [84]:
print(queries[:, -2:])

<tf.RaggedTensor [[b'George', b'Washington'], [b'weather', b'tomorrow'], [b'Goodnight']]>


### Indexing examples 3D ragged tensor

In [86]:
rt = tf.ragged.constant([[[1, 2, 3], [4]],
                         [[5], [], [6]],
                         [[7]],
                         [[8, 9], [10]]])
print(rt)
rt

<tf.RaggedTensor [[[1, 2, 3], [4]], [[5], [], [6]], [[7]], [[8, 9], [10]]]>


<tf.RaggedTensor [[[1, 2, 3], [4]], [[5], [], [6]], [[7]], [[8, 9], [10]]]>

In [87]:
print(rt[1])                        # Second row (2-D RaggedTensor)

<tf.RaggedTensor [[5], [], [6]]>


In [88]:
print(rt[3, 0])                     # First element of fourth row (1-D Tensor)

tf.Tensor([8 9], shape=(2,), dtype=int32)


In [89]:
print(rt[:, 1:3])                   # Items 1-3 of each row (3-D RaggedTensor)

<tf.RaggedTensor [[[4]], [[], [6]], [], [[10]]]>


In [90]:
print(rt[:, -1:])                   # Last item of each row (3-D RaggedTensor)

<tf.RaggedTensor [[[4]], [[6]], [[7]], [[10]]]>


`RaggedTensor`s supports multidimensional indexing and slicing, with one
restriction: indexing into a ragged dimension is not allowed. This case is
problematic because the indicated value may exist in some rows but not others.
In such cases, it's not obvious whether we should (1) raise an `IndexError`; (2)
use a default value; or (3) skip that value and return a tensor with fewer rows
than we started with. Following the
[guiding principles of Python](https://www.python.org/dev/peps/pep-0020/)
("In the face
of ambiguity, refuse the temptation to guess" ), we currently disallow this
operation.

## Tensor type conversion

The `RaggedTensor` class defines methods that can be used to convert
between `RaggedTensor`s and `tf.Tensor`s or `tf.SparseTensors`:

In [94]:
ragged_sentences = tf.ragged.constant([
    ['Hi'], ['Welcome', 'to', 'the', 'fair'], ['Have', 'fun']])

In [95]:
# RaggedTensor -> Tensor
print(ragged_sentences.to_tensor(default_value='', shape=[None, 10]))

tf.Tensor(
[[b'Hi' b'' b'' b'' b'' b'' b'' b'' b'' b'']
 [b'Welcome' b'to' b'the' b'fair' b'' b'' b'' b'' b'' b'']
 [b'Have' b'fun' b'' b'' b'' b'' b'' b'' b'' b'']], shape=(3, 10), dtype=string)


In [97]:
# Tensor -> RaggedTensor
x = [[-1, 3, -1, -1], [2, -1, -1, -1], [4, 5, 8, 9]]
print(tf.RaggedTensor.from_tensor(x, padding=-1))

<tf.RaggedTensor [[-1, 3], [2], [4, 5, 8, 9]]>


In [98]:
#RaggedTensor -> SparseTensor
print(ragged_sentences.to_sparse())

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [2 0]
 [2 1]], shape=(7, 2), dtype=int64), values=tf.Tensor([b'Hi' b'Welcome' b'to' b'the' b'fair' b'Have' b'fun'], shape=(7,), dtype=string), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [99]:
# SparseTensor -> RaggedTensor
st = tf.SparseTensor(indices=[[0, 0], [2, 0], [2, 1]],
                     values=['a', 'b', 'c'],
                     dense_shape=[3, 3])
print(tf.RaggedTensor.from_sparse(st))

<tf.RaggedTensor [[b'a'], [], [b'b', b'c']]>


## Evaluating ragged tensors

To access the values in a ragged tensor, you can:

1.  Use `tf.RaggedTensor.to_list()` to convert the ragged tensor to a
    nested python list.
1.  Use `tf.RaggedTensor.numpy()` to convert the ragged tensor to a numpy array 
    whose values are nested numpy arrays.
1.  Decompose the ragged tensor into its components, using the
    `tf.RaggedTensor.values` and `tf.RaggedTensor.row_splits`
    properties, or row-paritioning methods such as 
    `tf.RaggedTensor.row_lengths()` and `tf.RaggedTensor.value_rowids()`.
1.  Use Python indexing to select values from the ragged tensor.

In [100]:
rt = tf.ragged.constant([[1, 2], [3, 4, 5], [6], [], [7]])
print("python list:", rt.to_list())
print("numpy array:", rt.numpy())
print("values:", rt.values.numpy())
print("splits:", rt.row_splits.numpy())
print("indexed value:", rt[1].numpy())

python list: [[1, 2], [3, 4, 5], [6], [], [7]]
numpy array: [array([1, 2], dtype=int32) array([3, 4, 5], dtype=int32)
 array([6], dtype=int32) array([], dtype=int32) array([7], dtype=int32)]
values: [1 2 3 4 5 6 7]
splits: [0 2 5 6 6 7]
indexed value: [3 4 5]


## Broadcasting

Broadcasting is the process of making tensors with different shapes have
compatible shapes for elementwise operations. For more background on
broadcasting, see:

*   [Numpy: Broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)
*   `tf.broadcast_dynamic_shape`
*   `tf.broadcast_to`

The basic steps for broadcasting two inputs `x` and `y` to have compatible
shapes are:

1.  If `x` and `y` do not have the same number of dimensions, then add outer
    dimensions (with size 1) until they do.

2.  For each dimension where `x` and `y` have different sizes:

    *   If `x` or `y` have size `1` in dimension `d`, then repeat its values
        across dimension `d` to match the other input's size.

    *   Otherwise, raise an exception (`x` and `y` are not broadcast
        compatible).

Where the size of a tensor in a uniform dimension is a single number (the size
of slices across that dimension); and the size of a tensor in a ragged dimension
is a list of slice lengths (for all slices across that dimension).

### Broadcasting examples

In [101]:
# x       (2D ragged):  2 x (num_rows)
# y       (scalar)
# result  (2D ragged):  2 x (num_rows)
x = tf.ragged.constant([[1, 2], [3]])
y = 3
print(x + y)

<tf.RaggedTensor [[4, 5], [6]]>


In [102]:
# x         (2d ragged):  3 x (num_rows)
# y         (2d tensor):  3 x          1
# Result    (2d ragged):  3 x (num_rows)
x = tf.ragged.constant(
   [[10, 87, 12],
    [19, 53],
    [12, 32]])
y = [[1000], [2000], [3000]]
print(x + y)

<tf.RaggedTensor [[1010, 1087, 1012], [2019, 2053], [3012, 3032]]>


In [103]:
# x      (3d ragged):  2 x (r1) x 2
# y      (2d ragged):         1 x 1
# Result (3d ragged):  2 x (r1) x 2
x = tf.ragged.constant(
    [[[1, 2], [3, 4], [5, 6]],
     [[7, 8]]],
    ragged_rank=1)
y = tf.constant([[10]])
print(x + y)

<tf.RaggedTensor [[[11, 12], [13, 14], [15, 16]], [[17, 18]]]>


In [104]:
# x      (3d ragged):  2 x (r1) x (r2) x 1
# y      (1d tensor):                    3
# Result (3d ragged):  2 x (r1) x (r2) x 3
x = tf.ragged.constant(
    [
        [
            [[1], [2]],
            [],
            [[3]],
            [[4]],
        ],
        [
            [[5], [6]],
            [[7]]
        ]
    ],
    ragged_rank=2)
y = tf.constant([10, 20, 30])
print(x + y)

<tf.RaggedTensor [[[[11, 21, 31], [12, 22, 32]], [], [[13, 23, 33]], [[14, 24, 34]]], [[[15, 25, 35], [16, 26, 36]], [[17, 27, 37]]]]>


Here are some examples of shapes that do not broadcast:

In [105]:
# x      (2d ragged): 3 x (r1)
# y      (2d tensor): 3 x    4  # trailing dimensions do not match
x = tf.ragged.constant([[1, 2], [3, 4, 5, 6], [7]])
y = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'Unable to broadcast: dimension size mismatch in dimension'
1
b'lengths='
4
b'dim_size='
2, 4, 1


In [106]:
# x      (2d ragged): 3 x (r1)
# y      (2d ragged): 3 x (r2)  # ragged dimensions do not match.
x = tf.ragged.constant([[1, 2, 3], [4], [5, 6]])
y = tf.ragged.constant([[10, 20], [30, 40], [50]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'Unable to broadcast: dimension size mismatch in dimension'
1
b'lengths='
2, 2, 1
b'dim_size='
3, 1, 2
